In [5]:
import sys
!{sys.executable} -m pip install ultralytics opencv-python numpy


## Import thư viện

In [6]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
from functools import cmp_to_key

## Các đường dẫn và hệ số

In [7]:

MODEL_PATH = "./best.pt"
RGB_DIR = "./Public_data_task_3/Public_data/Public_data_train/rgb"
DEPTH_DIR = "./Public_data_task_3/Public_data/Public_data_train/depth"

VISUALIZATION_SAVE_DIR = "./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_visualization"
CROP_SAVE_DIR = "./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs"
os.makedirs(VISUALIZATION_SAVE_DIR, exist_ok=True)
os.makedirs(CROP_SAVE_DIR, exist_ok=True)


DEPTH_SCALE = 0.001 # Hệ số chuyển đổi giá trị depth sang mét

# Cấu hình logic xử lý và lựa chọn
FIXED_ROI_XYWH = (560, 150, 300, 330) # Tọa độ và kích thước của "Vùng Quan Tâm" (Region of Interest) cố định. Chỉ các bưu kiện có tâm nằm trong vùng này mới được xét.
HEIGHT_TOLERANCE_METERS = 0.005 # Ngưỡng chênh lệch chiều cao (5mm). Nếu hai bưu kiện chênh nhau ít hơn giá trị này, chúng được coi là cao bằng nhau.
MIN_MASK_PIXEL = 500 # Diện tích tối thiểu (tính bằng pixel) để một đối tượng được coi là một bưu kiện hợp lệ, giúp loại bỏ nhiễu.



## Các hàm càn dùng

### Hàm kiểm tra tâm (cx, cy) của kiện hàng có nằm trong vùng hợp lệ không

In [8]:
def is_center_in_roi(bbox_center, roi_xywh):
    cx, cy = bbox_center
    x, y, w, h = roi_xywh
    return (x <= cx < x + w) and (y <= cy < y + h)


### Hàm lấy giá trị depth và đổi nó sang mét

In [9]:
# Chỉ lấy giá trị pixel tại một điểm trên ảnh depth, kiểm tra xem nó có hợp lệ không (khác 0) và đổi giá trị đó ra mét bằng cách nhân với DEPTH_SCALE.
def get_depth_in_meters(cx, cy, depth_map, depth_scale):
    if cx < 0 or cy < 0 or cy >= depth_map.shape[0] or cx >= depth_map.shape[1]:
        return None
    z_val = depth_map[int(cy), int(cx)]
    if z_val == 0:
        return None
    return z_val * depth_scale

### Hàm load các ảnh đầu vào

In [10]:
def load_rgb_depth(rgb_path, depth_path):
    rgb = cv2.imread(rgb_path)
    depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
    if rgb is None: raise FileNotFoundError(f"RGB image not found: {rgb_path}")
    if depth is None: raise FileNotFoundError(f"Depth image not found: {depth_path}")
    if len(depth.shape) == 3: depth = depth[:, :, 0]
    return rgb, depth


### Hàm resize mask theo hình dạng của ảnh gốc

In [11]:
# resize mask theo hình dạng của ảnh gốc(do YOLO lúc nhận diện đã tự resize ảnh) 
def mask_resize_to_image(mask, target_shape):
    target_h, target_w = target_shape[0], target_shape[1]
    resized = cv2.resize(mask.astype(np.float32), (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    return (resized > 0.5).astype(np.uint8)

### Hàm tìm bounding box từ mask

In [12]:
def bbox_from_mask(mask):
    cnts, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return None
    c = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    return (x, y, w, h) if w > 0 and h > 0 else None

### Hàm tìm tâm của mask

In [13]:
# Tính toán tâm của bounding box
def center_of_bbox(bbox):
    x, y, w, h = bbox
    return (x + w / 2.0, y + h / 2.0)

### Hàm so sánh độ sâu hoặc xa robot giữa 2 bưu kiện

In [14]:
# So sánh để chọn bưu kiện
def compare_parcels_simplified(p1, p2):
    z1, z2 = p1['depth_meters'], p2['depth_meters']
    if abs(z1 - z2) > HEIGHT_TOLERANCE_METERS:
        return -1 if z1 < z2 else 1
    cy1, cy2 = p1['center'][1], p2['center'][1]
    return -1 if cy1 > cy2 else 1

## Tải mô hình

In [15]:
print("Loading YOLO model...")
model = YOLO(MODEL_PATH)
print("Model loaded successfully.")

Loading YOLO model...


Model loaded successfully.


## Vòng lặp chạy chính

In [16]:
for rgb_name in sorted(os.listdir(RGB_DIR)):
    if not rgb_name.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    base_name = os.path.splitext(rgb_name)[0]
    rgb_path = os.path.join(RGB_DIR, rgb_name)
    depth_path = os.path.join(DEPTH_DIR, rgb_name)
    try:
        rgb, depth = load_rgb_depth(rgb_path, depth_path)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        continue

    # Nhận tất cả các mask bưu kiện từ YOLO
    results = model.predict(rgb, verbose=False)
    valid_masks = []
    if results and results[0].masks is not None:
        raw_masks = results[0].masks.data.cpu().numpy()
        for raw in raw_masks:
            m = mask_resize_to_image(raw, rgb.shape)
            if np.count_nonzero(m) >= MIN_MASK_PIXEL:
                valid_masks.append(m)

    # Nếu YOLO không tìm thấy gì, bỏ qua
    if not valid_masks:
        print(f"No candidate objects found for {rgb_name}")
        continue

    # Xây dựng danh sách bưu kiện chỉ với depth
    all_candidates = []
    for mask in valid_masks:
        bbox = bbox_from_mask(mask)
        if bbox is None:
            continue
        cx, cy = center_of_bbox(bbox)
        z_meters = get_depth_in_meters(cx, cy, depth, DEPTH_SCALE)
        if z_meters is not None:
            all_candidates.append({
                "mask": mask, #Thêm mask vào để cắt sau này
                "bbox": bbox,
                "center": (cx, cy),
                "depth_meters": z_meters
            })

    # Lọc ứng viên trong ROI
    roi_candidates = [c for c in all_candidates if is_center_in_roi(c['center'], FIXED_ROI_XYWH)]

    if not roi_candidates:
        print(f"No parcels found inside the ROI for {rgb_name}")
        continue

    # Sắp xếp và tìm ứng viên tốt nhất
    roi_candidates.sort(key=cmp_to_key(compare_parcels_simplified))
    best = roi_candidates[0]
    
    # Tạo ảnh trực quan hóa
    x, y, w, h = best["bbox"]
    cx, cy = best["center"]
    Z = best["depth_meters"]
    
    vis_img = rgb.copy()
    cv2.rectangle(vis_img, (x, y), (x + w, y + h), color=(0, 255, 0), thickness=2)
    roi_x, roi_y, roi_w, roi_h = FIXED_ROI_XYWH
    cv2.rectangle(vis_img, (roi_x, roi_y), (roi_x + roi_w, roi_y + roi_h), color=(255, 0, 0), thickness=2)
    
    text = f"Center:({cx},{cy}) Depth Z:{Z:.2f}m"
    ty = y - 10 if y - 10 > 10 else y + h + 25
    cv2.putText(vis_img, text, (x, ty), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    vis_out_name = os.path.join(VISUALIZATION_SAVE_DIR, f"{base_name}_visualization.png")
    cv2.imwrite(vis_out_name, vis_img)
    
    print(f"Processed {rgb_name} -> Selected Parcel Center:({cx},{cy}), Depth: Z={Z:.4f} meters")

    # Cắt và lưu ảnh cho Giai đoạn 2 
    top_mask = best["mask"]
    
    rgb_crop = rgb[y:y+h, x:x+w]
    depth_crop = depth[y:y+h, x:x+w]
    mask_crop = top_mask[y:y+h, x:x+w]
    
    cv2.imwrite(os.path.join(CROP_SAVE_DIR, f"{base_name}_rgb_crop.png"), rgb_crop)
    cv2.imwrite(os.path.join(CROP_SAVE_DIR, f"{base_name}_depth_crop.png"), depth_crop)
    cv2.imwrite(os.path.join(CROP_SAVE_DIR, f"{base_name}_mask_crop.png"), mask_crop * 255)
    print(f" -> Saved cropped outputs for {base_name} to {CROP_SAVE_DIR}")


print("\nFinished processing all images.")



Processed 0000.png -> Selected Parcel Center:(616.0,386.5), Depth: Z=1.0640 meters
 -> Saved cropped outputs for 0000 to ./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs
Processed 0001.png -> Selected Parcel Center:(620.0,386.5), Depth: Z=1.0640 meters
 -> Saved cropped outputs for 0001 to ./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs
Processed 0002.png -> Selected Parcel Center:(596.0,285.5), Depth: Z=1.0500 meters
 -> Saved cropped outputs for 0002 to ./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs
No parcels found inside the ROI for 0003.png
No parcels found inside the ROI for 0004.png
Processed 0005.png -> Selected Parcel Center:(628.0,368.0), Depth: Z=1.0600 meters
 -> Saved cropped outputs for 0005 to ./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs
Processed 0006.png -> Selected Parcel Center:(628.0,350.0), Depth: Z=1.0330 meters